In [1]:
import os
import re
import numpy as np 
import pandas as pd 
from collections import Counter
import nltk

In [2]:
! pip install seaborn

In [3]:
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [4]:
import seaborn as sns
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from collections import Counter
import itertools
import operator
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Read text Data

In [5]:
Data = pd.read_json("Dataset.json", lines=True)
Data.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [6]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category           200853 non-null  object        
 1   headline           200853 non-null  object        
 2   authors            200853 non-null  object        
 3   link               200853 non-null  object        
 4   short_description  200853 non-null  object        
 5   date               200853 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [7]:
data = pd.read_csv('shakespeare.txt', sep="\n", header=None)
data.columns = ["text"]
data.head()

,text
0,THE SONNETS
1,by William Shakespeare
2,"From fairest creatures we desire increase,"
3,"That thereby beauty's rose might never die,"
4,"But as the riper should by time decease,"


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2158 entries, 0 to 2157
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2158 non-null   object
dtypes: object(1)
memory usage: 17.0+ KB


#### Checking For Na Values to be Removed

In [9]:
data.isna().sum()

text    0
dtype: int64

In [10]:
Data.isna().sum()

category             0
headline             0
authors              0
link                 0
short_description    0
date                 0
dtype: int64

#### Checking For Duplicates Values to be Removed

In [11]:
print("Number of Duplicats :",data.duplicated().sum())
print("Removing Duplicats ......")
data = data.drop_duplicates()
print("Number of Duplicats :",data.duplicated().sum())

Number of Duplicats : 2
Removing Duplicats ......
Number of Duplicats : 0


In [12]:
Data['text'] = Data['headline']+Data['short_description']
print("Number of Duplicats :",Data.duplicated(subset = 'text').sum())
print("Removing Duplicats ......")
Data = Data.drop_duplicates(subset = 'text')
print("Number of Duplicats :",Data.duplicated(subset = 'text').sum())

Number of Duplicats : 488
Removing Duplicats ......
Number of Duplicats : 0


## Prepare Text Data to make Vocab

In [13]:
from nltk.tokenize import word_tokenize

# 1- Remove Punctuation
def remove_punctuation(text):
  without_punctuation ="".join(i for i in text if i not in string.punctuation)
  return without_punctuation

# 2- Lower Case
def lower_case(text):
  return text.lower()

# 3- Tokenize Text
def tokenization(text):
  tokens = re.split('W+',text)
  return tokens

# 4- Remove Numbers
def remove_numbers(text):
  nums = re.compile(r'\d+')
  clean_text = nums.sub(r'',text)
  return clean_text

# Get Text Clean
def preprocess_text(text):
  tokens = word_tokenize(lower_case(remove_punctuation(remove_numbers(text))))
  return tokens

In [ ]:

Data['clean_text'] = Data['text'].apply(lambda x:preprocess_text(x))
data['clean_text'] = data['text'].apply(lambda x:preprocess_text(x))
Data['clean_text'].head()

In [ ]:
text1 = list(itertools.chain(*list(Data['clean_text'])))
# print("Text Length",len(text))
text2 = list(itertools.chain(*list(data['clean_text'])))
text = text1 + text2
print("Text Length",len(text))
print(text[:10])
# Text Length 17580
# Text Length 17581

#### Define My Vocab 

In [ ]:
vocab = set(text)
print("Length Of Vocab : ",len(vocab))

#### Get Probabity of each Word

##### First Count The Occerance of each word(Frequancy of the Word)

In [ ]:
def word_count(vocab):
    word_freq = {}
    word_freq = Counter(vocab)
    return word_freq

# print(word_count(["aya","hi","hello","aya"]))
# Counter({'aya': 2, 'hi': 1, 'hello': 1})
word_counter = word_count(text)
print("frequancy of [Okay] word = ",word_counter["okay"])

##### Then Calculate the Probability of occuaraing 

In [ ]:
# Words_freqs has to be dict words as keys , frequancy of word as value
def probability(words_freq):
    total = sum(word_counter.values())
    prob = {}
    for word in words_freq.keys():
        prob[word] = words_freq[word]/total
    return prob

prob = probability(word_counter)   
print("Probability of [Okay] word = ",prob["okay"])

##### Now we Clean our text Dataset and get distincit words => Vocab to calculate prob. of each word and any word outside these set will be recognized to be misspelled word so vocab had to be as big as possible

In [ ]:
# sentence has to be list of words
def misspeled_words(sentence,vocab = vocab):
    misspeled_word = []
    for word in sentence:
        if word not in vocab:
            misspeled_word.append(word)
    return misspeled_word


In [ ]:
sen = ["heel","home","hello","okayyy","the","off","aya","theeeeee"]
print("Wong words",misspeled_words(sen))

#### Statistical Model that we will impliment based on 4 stratgies for finding the correct word
#### 1- Delete Letter 2- Insert Letter 3- Replace Letter 4- Switching Two Letters

#### Delete Letter

In [ ]:
letters = 'abcdefghijklmnopqrstuvwxyz'

In [ ]:
def delete_letter(word):
    split = [(word[:i], word[i:]) for i in range(len(word))]
    delete = [s[0]+s[1][1:] for s in split]
    return delete


# print(delete_letter("home"))
# ['ome', 'hme', 'hoe', 'hom']

#### Insert Letter

In [ ]:
def insert_letter(word,letters = letters):
    split = [(word[:i], word[i:]) for i in range(len(word)+1)]
    insert = []
    for s in split:
        for l in letters:
            insert.append(s[0]+l+s[1])
    return insert

# print(len(insert_letter("home")))
# 130

#### Replace Letter

In [ ]:
def replace_letter(word,letters = letters):
    split = [(word[:i], word[i:]) for i in range(len(word))]
    replace = []
    for s in split:
        if len(s[1]) == 1:
            for l in letters:
                if l != s[1][0]:
                    temp = l
                    replace.append(s[0]+temp)
        elif len(s) > 1:
            for l in letters:
                if l != s[1][0]:
                    temp = l + s[1][1:]
                    replace.append(s[0]+temp)
        
    replace = list(set(replace))
    return replace

# print(len(replace_letter("home")))
# 100

#### Switch Letter

In [ ]:
def switch_letter(word):
    switch = []
    split = [(word[:i], word[i:]) for i in range(len(word))]
    for s in split:
        if len(s[1])>2:
            temp = s[0] + s[1][1] + s[1][0] + s[1][2:]
        elif len(s[1]) == 2:
            temp = s[0] + s[1][1] + s[1][0]
        elif len(s[1]) == 1:
            continue
        switch.append(temp)
    return switch

# print(switch_letter("home"))
# ['ohme', 'hmoe', 'hoem']

### Edit Only One Letter

In [ ]:
def edit_one_letter(word):
    
    delete = delete_letter(word)
    insert = insert_letter(word)
    replace = replace_letter(word)
    switch = switch_letter(word)
    
    one_edit = delete + insert + replace + switch
    one_edit = set(one_edit)
    
    return one_edit

# print(len(edit_one_letter("home")))
# 233

In [ ]:
def edit_two_letters(word):
    edit_two = []
    edit_one = edit_one_letter(word)
    for w in edit_one:
        edit_two.append(w)
        edit_two.extend(edit_one_letter(w))
    edit_two = set(edit_two)
    return edit_two

print(len(edit_two_letters("home")))        

In [ ]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [ ]:
take(4, best_words.items())

In [ ]:
def words_correction(word , prob = prob, vocab = vocab , n = 3):
    suggestion = []
    n_best = []
    best_words = {}
    if word in prob.keys():
        suggestion.append(word)
    for w in edit_one_letter(word):
        if w in prob.keys():
            suggestion.append(w)
    for w in edit_two_letters(word):
        if w in prob.keys():
            suggestion.append(w)
    for s in suggestion:
        best_words[s] = prob[s]
    best_words = dict( sorted(best_words.items(), key=operator.itemgetter(1),reverse=True))
    n_best = list(take(n,best_words))
    return n_best


print(correct_words("okayy"))
# ['okay', 'katy', 'kay']         

In [ ]:
print(type(correct_words("okayy")))

In [ ]:
word_counter['hella']

In [ ]:
def AutoCorrection(sentence, vocab = vocab, prob = prob):
    clean_sentence = preprocess_text(sentence)
    # print(clean_sentence)
    wrong_words = misspeled_words(clean_sentence)
    # print(wrong_words)
    # print(correct_words(wrong_words[0]))
    correct_words = {}
    for word in wrong_words:
        # print(type(word))
        _word = words_correction(word,n = 1)
        # print(_word)
        correct_words[word] = _word
    return correct_words

In [ ]:
sen = "I have acess to the lidrary"

print(AutoCorrection(sen))

In [ ]:
sen = "life is a diink annd lve is a druug"
print(AutoCorrection(sen))